
## Bring Your Own Algorithm to SageMaker
### Architecture of this notebook 


<img src="./images/part1.png" alt="architecture" width="800"/>


### 1. Training
a. [Bring Your Own Container](#byoc)

b. [Training locally](#local_train)

c. [Trigger remote training job](#remote_train)

d. [Test locally](#local_test)

### 2. Deploy EndPoint
[Deploy model to SageMaker Endpoint](#deploy_endpoint)

### 3. Build Lambda Function
a. [Construct lambda function](#build_lambda_function)

b. [Test lambda](#lambda_test)

### 4. Configure API Gateway

a. [Construct and setting api gateway](#api-gateway)

b. [Configure for passing binary media to Lambda Function](#binary-content)

c. [test api gateway](#test-api)


In [141]:
1+1

2

### BYOC (Bring Your Own Container) for Example Audio Classification Algorithm
<a name="byoc"></a>


* prepare necessry variables
using `boto3` to get region and account_id for later usage - ECR uri construction 

In [2]:
import boto3 

session = boto3.session.Session()
region = session.region_name
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
algorithm_name = "vgg-audio"

#### 3 elements to build bring your own container 
* `build_and_push.sh` is the script communicating with ECR 
* `Dockerfile` defines the training and serving environment 
* `code/train` and `code/serve` defines entry point of our container 

In [2]:
# 給最高權限
!chmod u+x *.sh

In [3]:
# import datetime
from datetime import datetime, timedelta, timezone
datetime.now(timezone(timedelta(hours=8)))

datetime.datetime(2021, 7, 16, 21, 36, 32, 288005, tzinfo=datetime.timezone(datetime.timedelta(0, 28800)))

In [7]:
# 這個不用重跑
!./build_and_push.sh
# import datetime
from datetime import datetime, timedelta, timezone
datetime.now(timezone(timedelta(hours=8)))

355444812467
us-west-2
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
base_img:763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04
1.6.0-gpu-py36-cu110-ubuntu18.04: Pulling from pytorch-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16445 sha256=7b80509c93a3bbb49f6fd4250231f0eecdda0f299ec54ccab33bca1d19c5a43e
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0_70ms6/wheels/63/17/7e/07f1c55a623c96ba9b291c5d2e4901afffb953b032c2470ca6


  Created wheel for librosa: filename=librosa-0.6.0-py3-none-any.whl size=1553494 sha256=2f5a127e87da2021fa0377fb7ce1c805a7e6cbacce6503cc1da437f8c63031a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0_70ms6/wheels/72/c0/f2/b00db6242414bafcbbca59c495a52a1bfd0574f29a7e498dc9
  Created wheel for audioread: filename=audioread-2.1.8-py3-none-any.whl size=23089 sha256=6d8df8fd0a1d003ed4def44d6bdf0ce458a2e69638275a9bad92074b99a2d8a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0_70ms6/wheels/1d/9a/c7/249a5daf5cb90d9786afaec371cba9dc43f04f916db5d1caff
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9693 sha256=d6b16a4c26664b545354866fdb113957cb0b76cfb90909d50805829fa6d71307
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0_70ms6/wheels/33/15/6e/df5f8336275e96e19599034a76f9cfd81c6ae15d2bf16c11ca
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=73b8f060804ab4fc6948597603066c126eaece2693d112ac100da518a00d1ebb
  Stored in 

    Uninstalling prompt-toolkit-3.0.8:
      Successfully uninstalled prompt-toolkit-3.0.8
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: jedi
    Found existing installation: jedi 0.18.0
    Uninstalling jedi-0.18.0:
      Successfully uninstalled jedi-0.18.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.0.1
    Uninstalling Jinja2-3.0.1:
      Successfully uninstalled Jinja2-3.0.1
  Attempting uninstall: cffi
    Found existing installation: cffi 1.14.5
    Uninstalling cffi-1.14.5:
      Successfu

Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [637 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [73.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [473 kB]
Get:12 http://archive.ubuntu.com/ubunt

Setting up python3 (3.6.7-1~18.04) ...
running python rtupdate hooks for python3.6...
running python post-rtupdate hooks for python3.6...
Setting up python3-cffi-backend (1.11.5-1) ...
Setting up libgfortran4:amd64 (7.5.0-3ubuntu1~18.04) ...
Setting up libogg0:amd64 (1.3.2-1) ...
Setting up libblas3:amd64 (3.7.1-4ubuntu1) ...
update-alternatives: using /usr/lib/x86_64-linux-gnu/blas/libblas.so.3 to provide /usr/lib/x86_64-linux-gnu/libblas.so.3 (libblas.so.3-x86_64-linux-gnu) in auto mode
Setting up libvorbis0a:amd64 (1.3.5-4.2) ...
Setting up liblapack3:amd64 (3.7.1-4ubuntu1) ...
update-alternatives: using /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3 to provide /usr/lib/x86_64-linux-gnu/liblapack.so.3 (liblapack.so.3-x86_64-linux-gnu) in auto mode
Setting up libflac8:amd64 (1.3.2-1) ...
Setting up libvorbisenc2:amd64 (1.3.5-4.2) ...
Setting up python3-numpy (1:1.13.3-2ubuntu1) ...
Setting up libsndfile1:amd64 (1.0.28-4ubuntu0.18.04.1) ...
Setting up python3-soundfile (0.10.0-1) ...

datetime.datetime(2021, 7, 16, 15, 16, 24, 729545, tzinfo=datetime.timezone(datetime.timedelta(0, 28800)))

In [4]:
!cat Dockerfile

ARG BASE_IMG=${BASE_IMG}
FROM ${BASE_IMG} 
COPY code/* /
RUN pip install -r requirements.txt --no-cache-dir 
RUN pip install numba==0.48.0 --user
RUN apt-get update -y && apt-get install python3-soundfile -y  && rm -rf /var/lib/apt/lists/* && apt-get clean
WORKDIR /
ENV PATH="/:${PATH}"





In [5]:
!cat build_and_push.sh

#!/bin/bash

# The name of our algorithm
algorithm_name=vgg-audio

#cd container
# get information - account and region, required by ECR https://aws.amazon.com/ecr/
account=$(aws sts get-caller-identity --query Account --output text)
echo $account
# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}
echo $region


# derive fullname of docker image 
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker TensorFlow image
$(aws ecr get-logi

* construct image uri by account_id, region and algorithm_name

In [6]:
image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}"
image_uri

'355444812467.dkr.ecr.us-west-2.amazonaws.com/vgg-audio'

* prepare necessary variables/object for training 

In [7]:
import sagemaker 
session = sagemaker.session.Session()
bucket = session.default_bucket()

In [9]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

# s3_path = f"s3://{bucket}/data/competition"
# s3_path

arn:aws:iam::355444812467:role/service-role/AmazonSageMaker-ExecutionRole-20210702T211675


### Dataset Description - 

Dataset used in this workshop can be obtained from [Dog Bark Sound AI competition](https://tbrain.trendmicro.com.tw/Competitions/Details/15) hold by the world leading pet camera brand [Tomofun](https://en.wikipedia.org/wiki/Tomofun). The url below will be invalid after workshop. 

In [12]:
# s3://tomofun-audio-classification-yianc
# data/data.zip
# !wget -O data.zip https://tinyurl.com/ydzvotsd
# !unzip  train.zip     
# !aws s3 cp --recursive ./input/data/competition/ $s3_path

### Train model in a docker container with terminal interface 
<a name="local_train"></a>

* start container in interactive mode
```
IMAGE_ID=$(sudo docker images --filter=reference=vgg16-audio --format "{{.ID}}")
nvidia-docker run -it -v $PWD:/opt/ml --entrypoint '' $IMAGE_ID bash 
```
* train model based on README.md
```
python train.py --csv_path=/opt/ml/input/data/competition/meta_train.csv --data_dir=/opt/ml/input/data/competition/train --epochs=50 --val_split 0.1
```

In [13]:
from datetime import datetime

In [10]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "audio-{}".format(str(int(timestamp))) 
job_name

'audio-1626442613'

### Start SageMaker Training Job
<a name="remote_train"></a>
* sagemaker training jobs can run either locally or remotely 

In [9]:
# mode = 'local'
# if mode == 'local':
#     csess = sagemaker.local.LocalSession()
# else:    
#     csess = session

# print(csess)
# estimator = sagemaker.estimator.Estimator( 
#                         role=role,
#                         image_uri=image_uri,
#                         instance_count=1,
#                         instance_type='local_gpu',
# #                         instance_type='ml.p3.8xlarge',
#                         sagemaker_session=csess,
#                         volume_size=100, 
#                         debugger_hook_config=False
#                    )

In [18]:
# estimator.fit(inputs={"competition":s3_path}, job_name=job_name)

In [19]:
# estimator.model_data

In [10]:
# model_s3_path = estimator.model_data

# !aws s3 cp $model_s3_path . 
# !tar -xvf model.tar.gz
# !mkdir -p model 
# !mv final_model.pkl model/

### Test Model Locally 
<a name="local_test"></a>

* start container in interactive mode
```
IMAGE_ID=$(sudo docker images --filter=reference=vgg16-audio --format "{{.ID}}")
nvidia-docker run -it -v $PWD:/opt/ml --entrypoint '' $IMAGE_ID bash 
```
* test model based on README.md
```
python test.py --test_csv /opt/ml/input/data/competition/meta_train.csv --data_dir /opt/ml/input/data/competition/train  --model_name VGGish --model_path /opt/ml/model --saved_root results/test --saved_name test_result
```

### Deploy SageMaker Endpoint  
<a name="deploy_endpoint"></a>


In [11]:
# predictor = estimator.deploy(instance_type='ml.p2.xlarge', initial_instance_count=1, serializer=sagemaker.serializers.IdentitySerializer())
# predictor = estimator.deploy(instance_type='local_gpu', initial_instance_count=1, serializer=sagemaker.serializers.IdentitySerializer())

In [12]:
# endpoint_name = predictor.endpoint_name

### You can deploy by using model file directly  

The Source code is as below. we can use model locally trained to deploy a sagemaker endpoint 

#### get example model file from s3 
```
source_model_data_url = 'https://tinyurl.com/yh7tw3hj'
!wget -O model.tar.gz $source_model_data_url

MODEL_PATH = f's3://{bucket}/model'
model_data_s3_uri = f'{MODEL_PATH}/model.tar.gz'
!aws s3 cp model.tar.gz $model_data_s3_uri
```


#### build endpoint from the model file 
```
import time 

mode = 'remote'

if mode == 'local':
    csess = sagemaker.local.LocalSession()
else:
    csess = session

model = sagemaker.model.Model(image_uri, 
                              model_data = model_data_s3_uri,
                              role = role,
                              predictor_cls = sagemaker.predictor.Predictor,
                              sagemaker_session = csess)

now = datetime.now()
timestamp = datetime.timestamp(now)
new_endpoint_name = "audio-{}".format(str(int(timestamp))) 
object_detector = model.deploy(initial_instance_count = 1,
                               instance_type = 'ml.p2.xlarge',
#                                instance_type = 'local_gpu',
                               endpoint_name = new_endpoint_name,
                               serializer = sagemaker.serializers.IdentitySerializer())
```

In [11]:
image_uri

'355444812467.dkr.ecr.us-west-2.amazonaws.com/vgg-audio'

In [119]:
model_2.name = 'zz'

In [117]:
1+1

2

In [104]:
model_data_s3_uri

's3://sagemaker-us-west-2-355444812467/model/model_0716_liyu.tar.gz'

In [ ]:
model_data_s3_uri

## 直接更新!!!

In [125]:
model_2 = sagemaker.model.Model(image_uri, name = 'zz',
                              model_data = f's3://sagemaker-us-west-2-355444812467/model/{model_targz}',
                              role = role,
                              predictor_cls = sagemaker.predictor.Predictor,
                              sagemaker_session = csess)

In [126]:
model_2.name

'zz'

In [142]:
print(datetime.now(timezone(timedelta(hours=8))))
model_targz = 'model_0725_liyu.tar.gz'
# model_name = 'vgg-audio-2021-07-16-13-50-04-174'
model_2 = sagemaker.model.Model(image_uri,
                              model_data = f's3://sagemaker-us-west-2-355444812467/model/{model_targz}',
                              role = role,
                              predictor_cls = sagemaker.predictor.Predictor,
                              sagemaker_session = csess)

now = datetime.now()
timestamp = datetime.timestamp(now)
new_endpoint_name = "audio-{}".format(str(int(timestamp))) 
object_detector = model_2.deploy(initial_instance_count = 1,
                               instance_type = 'ml.p2.xlarge',
#                                instance_type = 'local_gpu',
                               endpoint_name = new_endpoint_name,
                               serializer = sagemaker.serializers.IdentitySerializer())
print(datetime.now(timezone(timedelta(hours=8))))
# model_2.name = 'vgg-audio-2021-07-15-16-14-07-334'
# 更新model再跑
new_detector =  sagemaker.predictor.Predictor(endpoint_name = endpoint_name) 
new_detector.update_endpoint(model_name=model_2.name, initial_instance_count = 1,
                                instance_type = 'ml.p3.2xlarge')
print(datetime.now(timezone(timedelta(hours=8))))

2021-07-17 12:14:06.332961+08:00
-----------------------!2021-07-17 12:25:39.244282+08:00
---------------------!2021-07-17 12:36:12.517053+08:00


In [ ]:
'./Final_Training_Dataset/train/train_00001.wav'

In [116]:
import json 
file_name = './Final_Training_Dataset/train/train_00001.wav'
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
results = new_detector.predict(b)
detections = json.loads(results)
print(detections) 

{'label': 0, 'probability': [0.9903320670127869, 0.0001989333686651662, 0.0007428666576743126, 0.00016775673429947346, 3.222770828870125e-05, 3.8894515455467626e-05, 0.0003140326589345932, 0.0003089535457547754, 3.332734195282683e-05, 0.007830953225493431]}


In [115]:
import json 
file_name = './Final_Training_Dataset/train/train_00001.wav'
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
results = object_detector.predict(b)
detections = json.loads(results)
print(detections) 

{'label': 0, 'probability': [0.9903320670127869, 0.0001989333686651662, 0.0007428666576743126, 0.00016775673429947346, 3.222770828870125e-05, 3.8894515455467626e-05, 0.0003140326589345932, 0.0003089535457547754, 3.332734195282683e-05, 0.007830953225493431]}


In [114]:
object_detector.predict('./Final_Training_Dataset/train/train_01500.wav')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>audioread.exceptions.NoBackendError // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not by accident trigger a request to /favicon.ico which might
         change the application state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 140135917314344,
          CONSOLE_MODE = false,

In [113]:
new_detector.predict('./Final_Training_Dataset/train/train_01500.wav')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>audioread.exceptions.NoBackendError // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not by accident trigger a request to /favicon.ico which might
         change the application state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 140135917224120,
          CONSOLE_MODE = false,

In [112]:
# 1228
from datetime import datetime, timedelta, timezone
print(datetime.now(timezone(timedelta(hours=8))))

2021-07-17 00:37:24.916723+08:00


In [ ]:
model_0717_liyu_0.tar.gz

In [109]:
model.name

'vgg-audio-2021-07-16-13-50-04-174'

In [68]:
endpoint_name

'audio-20210716'

In [107]:
model.name

'vgg-audio-2021-07-16-13-50-04-174'

In [69]:
# 更新model再跑
new_detector =  sagemaker.predictor.Predictor(endpoint_name = endpoint_name) 
new_detector.update_endpoint(model_name=model.name, initial_instance_count = 1,
                                instance_type = 'ml.p3.2xlarge')

-------------------!

In [70]:
# 1050
from datetime import datetime, timedelta, timezone
print(datetime.now(timezone(timedelta(hours=8))))

2021-07-16 22:59:45.579358+08:00


In [13]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs        31426652       72  31426580   1% /dev
tmpfs           31437352        8  31437344   1% /dev/shm
/dev/xvda1     108305356 89591192  18613916  83% /
/dev/xvdf      515535104 53854976 439623092  11% /home/ec2-user/SageMaker


In [14]:
# model_data_s3_uri = 's3://sagemaker-us-west-2-355444812467/model/model_0715_liyu.tar.gz'
model_data_s3_uri = 's3://sagemaker-us-west-2-355444812467/model/model_0716_liyu.tar.gz'
model_data_s3_uri

's3://sagemaker-us-west-2-355444812467/model/model_0716_liyu.tar.gz'

In [16]:
# endpoint_name

In [25]:
# 自己建立Deploy
mode = 'remote'

if mode == 'local':
    csess = sagemaker.local.LocalSession()
else:
    csess = session

model = sagemaker.model.Model(image_uri, 
                              model_data = model_data_s3_uri,
                              role = role,
                              predictor_cls = sagemaker.predictor.Predictor,
                              sagemaker_session = csess)

In [26]:
image_uri

'355444812467.dkr.ecr.us-west-2.amazonaws.com/vgg-audio'

In [27]:
model_data_s3_uri

's3://sagemaker-us-west-2-355444812467/model/model_0716_liyu.tar.gz'

In [30]:
# from datetime import datetime, timedelta, timezone

new_endpoint_name = 'audio-20210716'
# print(datetime.now(timezone(timedelta(hours=8))))

# new_detector =  sagemaker.predictor.Predictor(endpoint_name = new_endpoint_name) 
# new_detector.update_endpoint(model_name=model.name, initial_instance_count = 1,
#                                 instance_type = 'ml.p2.xlarge')

# print(datetime.now(timezone(timedelta(hours=8))))

In [20]:
# now = datetime.now()
# timestamp = datetime.timestamp(now)
# new_endpoint_name = "audio-{}".format(str(int(timestamp))) 
# new_endpoint_name

In [31]:
object_detector = model.deploy(initial_instance_count = 1,
                               instance_type = 'ml.p3.2xlarge',
#                                instance_type = 'local_gpu',
                               endpoint_name = new_endpoint_name,
                               serializer = sagemaker.serializers.IdentitySerializer())

-------------------!

In [103]:
model.name 

'vgg-audio-2021-07-16-13-50-04-174'

In [33]:
from datetime import datetime, timedelta, timezone

print(datetime.now(timezone(timedelta(hours=8))))

new_detector =  sagemaker.predictor.Predictor(endpoint_name = new_endpoint_name) 
new_detector.update_endpoint(model_name=model.name, initial_instance_count = 1,
                                instance_type = 'ml.p2.xlarge')

print(datetime.now(timezone(timedelta(hours=8))))


2021-07-16 00:18:52.444700+08:00
-----------------------!2021-07-16 00:30:26.244070+08:00


In [32]:
model.name # 'vgg-audio-2021-07-16-13-50-04-174'

'vgg-audio-2021-07-16-13-50-04-174'

In [102]:
model.name

'vgg-audio-2021-07-16-13-50-04-174'

In [33]:
object_detector.endpoint_name

'audio-20210716'

### You can also update endpoint based on following example code 

```
new_detector =  sagemaker.predictor.Predictor(endpoint_name = endpoint_name) 
new_detector.update_endpoint(model_name=model.name, initial_instance_count = 1,
                                instance_type = 'ml.m4.xlarge')
```

In [34]:
import json 
file_name = "./train_data/train_00002.wav"
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
results = object_detector.predict(b)
detections = json.loads(results)
print(detections) 

{'label': 0, 'probability': [0.9513887763023376, 0.00023666038759984076, 0.04708951339125633, 9.738496009958908e-05, 6.840967398602515e-05, 5.344108558347216e-06, 4.340481973486021e-05, 0.00042902593850158155, 1.1892406291735824e-05, 0.0006295717321336269]}


In [35]:
file_name = "./train_data/train_00090.wav"
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
results = object_detector.predict(b)
detections = json.loads(results)
print(detections) 

{'label': 0, 'probability': [0.9998981356620789, 2.4996995762194274e-06, 4.887235627393238e-05, 1.990066266444046e-06, 3.2699779239919735e-07, 2.4871712867025053e-07, 1.0537002708588261e-05, 1.0949130228254944e-05, 1.7080250472645275e-05, 9.29264206206426e-06]}


In [ ]:
 [0.9998981356620789, 2.4996995762194274e-06, 4.887235627393238e-05, 1.990066266444046e-06, 3.2699779239919735e-07, 2.4871712867025053e-07, 1.0537002708588261e-05, 1.0949130228254944e-05, 1.7080250472645275e-05, 9.29264206206426e-06]

### Create Lambda Function 
<a name="build_lambda_function"></a>

In [36]:
import time 
iam = boto3.client("iam")

# role_name = "AmazonSageMaker-LambdaExecutionRole"
# assume_role_policy_document = {
#     "Version": "2012-10-17",
#     "Statement": [
#         {
#           "Effect": "Allow",
#           "Principal": {
#             "Service": ["sagemaker.amazonaws.com", "lambda.amazonaws.com"]
#           },
#           "Action": "sts:AssumeRole"
#         }
#     ]
# }

# create_role_response = iam.create_role(
#     RoleName = role_name,
#     AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
# )


# # Now add S3 support
# iam.attach_role_policy(
#     PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
#     RoleName=role_name
# )

# iam.attach_role_policy(
#     PolicyArn='arn:aws:iam::aws:policy/AmazonSQSFullAccess',
#     RoleName=role_name
# )

# iam.attach_role_policy(
#     PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
#     RoleName=role_name
# )
# time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

# lambda_role_arn = create_role_response["Role"]["Arn"]
# arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole
lambda_role_arn = "arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole"
print(lambda_role_arn)

arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole


In [37]:
bucket

'sagemaker-us-west-2-355444812467'

In [38]:
%%bash -s "$bucket" 
cd invoke_endpoint 
zip -r invoke_endpoint.zip  .
aws s3 cp invoke_endpoint.zip s3://$1/lambda/

updating: lambda_function.py (deflated 51%)
upload: ./invoke_endpoint.zip to s3://sagemaker-us-west-2-355444812467/lambda/invoke_endpoint.zip


In [39]:
import os
cwd = os.getcwd()
!aws lambda create-function --function-name invoke_endpoint --zip-file fileb://$cwd/invoke_endpoint/invoke_endpoint.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role $lambda_role_arn


An error occurred (ResourceConflictException) when calling the CreateFunction operation: Function already exist: invoke_endpoint


In [40]:
endpoint_name = object_detector.endpoint_name
bucket_key = "audio-demo"
variables = f"ENDPOINT_NAME={endpoint_name}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name invoke_endpoint --environment "$env"


{
    "FunctionName": "invoke_endpoint",
    "FunctionArn": "arn:aws:lambda:us-west-2:355444812467:function:invoke_endpoint",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 529,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-07-16T14:04:17.342+0000",
    "CodeSha256": "Oa3KNKU1lrPgGJzpg1Eot6grKw87LJkjGolYHLjuGPE=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "ENDPOINT_NAME": "audio-20210716"
        }
    },
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "2d97fd49-7e49-4037-89c2-76b0b9a69496",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


### Test Material 
<a name="lambda_test"></a>
```
{
  "content": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRUZGRgYGBgYGBoYGBoYGBgYGhgZGRgYGBgcIS4lHB4rIRgYJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJCQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAMMBAgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAEAAIDBQYBB//EADkQAAEDAgQEBAMGBgIDAAAAAAEAAhEDIQQSMUEFBlFhEyJxgTKhsUKRwdHh8BQVUmJygiPxB5Ky/8QAGQEAAwEBAQAAAAAAAAAAAAAAAAECAwQF/8QAJBEAAgICAwACAgMBAAAAAAAAAAECESExAxJBE1EiMgRhgXH/2gAMAwEAAhEDEQA/AABjTG6aKyAZWC67EhcDbDsWLn2QtWoVC3EAqUEKbYdgd9KdUFX4c0o6u6NEmGVomIqTw0dENW4SDstA4BDuTUmBS0uDiVLU4SNCrunAXKjgm5sADDcHaRGUKj4hwlzXWG61rKkBQYhwKcZtOxUYd+FcNk0UndFr3YVpSfgGwtPlY8mNLT0SAK1zeHN6JzeGN6I+VBZksvUKMha+rwodFEzgwnRC5UKyhZmy2BQtQkm62/8ALQGqlxXB5d5QlHkVgmZ9JXL+COCHfwp4WqnFjtFckjv5a9NdgHBO0FoDSUpw7ui4aThsU7QyNJSCiei4WEbItAMBSKSUIASSRCSAEkkkgDXGuVzxVASmBy5EhBza6kOKIQTXJr3lNxFQd/FSnnGQFVhyeRZLqJhYxqe3EKtIU9JKgDhXSa8ygy6EvGRQFk+quEygGVC4wFe4HglR0F8U2/3fEfRv5otIqKlJ0kQ4PDh13TlHTUnoFd/wdF7LMLSdHAkkHvJgqQiixuUbD7h17pYbiDG7SD1tH5LGUm3g7YcUVGmsgNPgDyfLUZHW/wAwmYzg1anfLnb/AFMkx6iJC0+EOe9o7z+KPZiabCJqNHUG6PkfoPgi9HnLnwuU6q9QxWDoV2Fr8pkGHNABaToQUPgeWMMwAtYXnWXmflp8k1NUYvgdmCbRe4SGOI6wYXKZG4Xrjn5GEaNANsrdIVZR5awrpc5mYuuczjN+gBgeyblHwn4JHmtVzUK9jVveJcq4UuysL6ZO4OZs9CHfmlheWcOxozAVXaOLiW/+rZshTig+CVmDZRaVFWwjdoXoLKGHpkinTDHdYLj9529E19U7NbHVoAjuRCPmXhov4rrLPPsNwd9Qwxhd6Cw9ToEY/k6pEuLG9nOP1AIWsdiHTrPr+Sc7ENcMpspfO/DSP8WK2zB47l+pREvZDTo5pDmH/YWB7FVVTBjovTWPyTo5hEOY64c3cELMce4c2k/yXY8B7JuQD9knqD+C0hy9jDm4emVoyD+HjonNwQ6K2cJ2SFNbdznKl3D5GiFOAutHkEIOsxNTHZWfwASRmUrqfYLDGUJTH0YKJpPSqGVnYwRzEwMRLmKNzYRYEMQkSpMspgYiwIi1PYYTnNUYclQrHuN04XSaJTiIQInw9QsJe0eYAlvr+cSh8XzBVcfjOnX8FzOg8XQnzN9x36hHVXk24pUqHjiT+pU9HiD5EEkuP00CqQYN0XhjLgRtum4pG/Z/ZccR5kfTbkYb6SCbnc+nRZ3+ZVnuBL3EylisI74j6W2RXC8KHvAPwi59AqqEY3RPaUpbwekcv8ScWMzAxb1J6wt3hXktDoi2i8xw/GKVDLmgWnsBt7q84TzjTqmA6DMRtA3XG00rrB0YeLNgAXlzSYGVxI9GkoWhXd11+SJ5ZqNqB5I1zN9oVFUxDWPex5gA2J0hS1UVL7sE/wAnEvThcw819x++qz/N1Ooyk57ZMAzHyPqoqfMtNj8hqNE6EuhaV9dlRmUwQ6x/RKsZQ7aPH8BzkWuAcTGkm/4LT8O4uysYY4B/TZ3p+SxfNPAhTqPDLQ4/mFV8IbUaZBjKZB6EdCuh8UJR7QZHyyUqkj1l7JvEOF+nuFVcRxTWAuecsW/6R2D4wx1NheQX5fNpqLGR+9VVcb4hScIDG37yfumAsEldGzdK7HYfEB1Jry17swJADg2GCwJsTJIMe3VVfF6pc5l5bkGWdQNCD3kIetxBxblFhA+URHawQufSSt4xrJx83KpKkyVRl9106KGbrQ5iYlNc0LhNlHmQJnMgSTl1FsQxlRPc9KrRhOpM7JlCYDqlUKMY2yGfR1QDQM1yTinChdJ1IpEkVRQEoh7CozQKYDfEXDUXXUCuHDlNIdHJXXFMa0p5TBCYAdQnuaFxgXXBSxj2MEQpOG4bK5xm0W97/gowYRVB3lJlTLRpwvNFdj8F4jrHzDrpCs+W+WanjDzNiL7g9h30+9Op4HP5i8NHpeB6LX8Ir0KNMPfNogadSJ7mFPeTj1R0qMU+zN5ytg8gaAIDRfue6B5q4CXguYGk31EiFQYPnOS4spuIsBDgIJMCVpeX+YxiC6i9jvEa0lwdAcWyBItfULX4bhTM/lqdo8pfyjUc5xcXQTldo2zbi3QbEL0Hlrh7WMaA5xDRAzOLr+6t+NcGNsroB+1Egjo4dUM0CiweabWgRJ9FzzlJvq/DaPWrj6ed89tjEubPlLWu++fnqsy58aLU84DNUncMbJtrLjH3ELLuYtOJLqc/M25UJlSVMAhskKRj1ZiSOXIXQ5LMihCkppUkrhRQUNldyroCdMIoBmQpKXOEk6YqQhVlPY8KLw4TgEgoKZXCe9wIQDmFRlzkUCDMi5lUdOoVIXooZwNvdEeGIQbqhldNYwl1AkewJll2kZ1XKohOgGeCCoX0EQEg2UACikniknudCZnQA0005php/f73XAV10EaxEEe179kpLBUHUkF4WuAWg2uB6g6fijuNYx9TDupC0GZGttB81VcWwrm0i9mkNe0ja6bw/iQqAtLocR5mnQ9x1CcI1+RvOVqjPYZxaS3PAMAwT5TJE+0fNek/+I8C5uIdWNUuY1mT+0ucQXBpJkgR2uvOcVSptcZdfNcC8X6jZej8kcwUKNFzc7Z11yxGgJN102YHp3NXFGUcK+pJMRA1MyNFh38S8anTqmxJyR/cQTMeyp+LcXdxF4oUSW0ZzPfoXwW/1XiJ7kxoNdlyvwSS177MYZYztoCfYBcvNBykkjfjkoxyUXH+DEMNR5Azhoa37XwgG+5Xn+Jplji06g/9Fem88uL8Y0D4W0wBqbkmbfcvPeOUyHj/AB1PqUsRfVCmrj2YCxdKa0JwKujnOTCRK48pjE6AJY+y4EwBdDUBY8PSe6UxwhNzpCsV0kvEC4jsItxQldGHurQUhoFK3DBTZp1KR7OygqM7K6xVMdFBTpgo7B1KtrCphRI1CsG0bpzxKVh1K4YeV11EKwDAntoAosVFO+idkvDO6tBSE3ThRBTsKKhrF1jJVwcM0KIUgEdvA6lX4EqB1FXooKF2HCaYqKplJONIaQrNmGB0RLMFNgJhDd4KSoi4dRz0sr7QS2+mUjT0us3U4Pkqmm6A5zXZO9iT8gfvWqxOJyQ0jaD0lA8QoGs1rmOAqMByE76QAdiri+rLu2ZkcEOkbj63Wm4TwSmKbxiDkDv+Ng3L3tcWH08s+yhwPNLaYIq4QmoCIc13lJ7tIstNw59TFllSrTbSptcHBhIzPI0uR0kbfEffZtGlxStGo5Q5dpYZjAYLy3M+e9hbQDULR1MSxj8jdSJPtsqHhwqOqOc9rhmFyLgCIG+yu6GFp2MS7QTss1oybtlPx3AzUa/KYy3cBN9gs5zByz4tPMweZoJZ/deXMPrt3XqXgiIOioeKUgHQ0RfQad1hzRcfyTNYSUl1Z4ezCHQjRNfhey2vNHDPDf4jR5HmT0Dzcj3uVngJKcZWrMJR6uipOGURoFW9SneEhTA1VWTRU+GUvDKtHsErhYITsKKt7ZUIZdW3gjRdZhQCkKiq/h+xSV/4DUkBQeydVwYqLJ+HqWQ1dt1kW2OqvlDiWlTNlJ7eqdBY9olNewhcBI0Uk9U6GRN1RTBZQ09ZTnPG6SQqGVmrlJymzghQYcyYVD0Odqk9kXUlamBdNAzeil0sieBuddYQpXUQdCh3UyClGSehJ4Jm0byp8HZ+utvyQZqGLIjhzC57fWVcbspBGNwuYEbqh8CPKTBBsfpC1WIYZsEFiKDHxnEEHUReFpJDRDgwx13tBdu4tABP7hX/AA54eHUnR8JyWgAi7dFFgeDtPwmw7D9J2ROH4U5rpDmxmsTMiDe4uNxupbaKikyy4RiX5AxpfrveOy1OEpwJce/Qqnw4ywA6ZvbtrfZWmHcSeoVRFIsWGUPxDCBzZA8wuETSTqxsfRXOKlFpkxbTwYzjODD6b2ET5SW/5C4XnIIb6r07HOgx2K874jhQDmA66dVxccqbRvyxtdgCq64JTKhulVqhOosButbOehGFG6l0UtWn0TaQKOyYtkGUyngKQtukXABJyoUmkR5ikn5uySnuT2D3EBMIBUlOnm12UlKnJ00Qk0WlggZTJcOinLBOiKFMLjqZBkXQUC1RA0QzKbiVYam6Ip5WppCor6dM9FFVwpVg/wCKdihnvgmUk2NkDKDhpom5YMoh2IMABRYhx2CdhSHsEi6ieSNkwVSLFEZ5Gil2weUDsc4qSgSTB0RTQMkkKNjxeE44EkkQVG3MKfhNQh8dj9FD4l0qZh2b1Vp07GlbLx7utwi8Ng21D8J/1k/NC8NcH6wJ6q+/k5DbSJ6GxXRtD0GcNwQZ5XAjYZomehg2Vo2iwGIF/wB7rP4Z+TyukHrNj6zoiGVMzhc/eoWBsKxNZjHEQQNo2JROG4nSFhmn/FR4/DtIzbIdlNpNvdVoRo6FYOEt0TsTUAbqqrDPyugaKTHPdIt+ScpVFhGNsqOKN8xOyyHEsoYT0d9bfitjxIyDbQLG8bHkcB2+oleev2OmX6FFVc03hQvrCLJswLqNlOdFvSOR5HGuUn4qEQWNAAUL6QzRsmkhJEuGqA6rkCUjQACYwQjAMfASTfDHVdRQYLV1AsGZS4Zh1K7XJMAqdlOQob8G/oY8rrRNgoatM6dE6gYN0k3YqZ12G1umuGVhUue5hQvqzbYaqrKqkCUMzutlPVph0HfdE0y0adLqFo887FIWyM0w1wJ0TKwzOFrKxq0xv7KAggTCd+DS8IG4e8kWClZTgF0WRAe0sJNioXPLmgDRCCiBtbMCIsmsokbJ9NwY2SLSp6r9C3dF+j3kHdoLaJuNgttspGVQTA21UmJALHAawUWEcMfy/wCV2ZzrDYR8zqvQsFjmPYBoR7ELFcvYIvYSW7fhCt8Dw17I809Jm/a6h8soukdKgpLJd43CAtzWWcw9fLULdpWjq4kMovzkCAd1g34uX5mrpcrSZjVOjWcW4jDMo1gFZ6hxZwdrH5LtWqXAON7KnxVTKZCbb2SjUt4kYDswgant6rmH44+q8AAwPU/UarJsxudjmt+KQ33NgfqtfwSm1rAGiwGvU3ue65+abqkb8UVst8U2Wz1aZWO4405JA+IgBbl7paFhOZJDwxn2Jn3/AEWaWQm/xaM/icNbulQow2US9zgbiU1rXAXFlon9nKo0DvbFkOHkOhWzKQ33TBgW5rXVJjoHLbXTcRTltkY6gu0sOfZFhTsqfAPVJW2XskixUi0xNNoEoZmINxsEXVF9fZcr4fSNwpoojw+YDM4WOiY+mbuUhaWsh5k7BDGsbNE7J2BPQpF11ABE23hWAEXE2GnVNa+ZJHeEMAdrCRA1KjewsIEI6k9okxB2Ujhm9OqT+gKr+Jc4y4aaIwMloKDreVxbsiWYkmGgbITBsixFIWaTGYp4pgGBpomPZL2z9ko6vVAdLR2TDwEdQtBvdReDY9Roj2sIAdsUI6pBch6Ahw9K56zdTGBMCbGfuTsKYc4kahPADWGR5nAj71KQKrL7l+uMhbpb6K8wjg45Tvp6rz7g+LLX/rZbHD1SRLetuxSkns6lnBTc+4ktc2ju5oe70kgfQrLMqWhaXn6jm8LEaOA8CoDsfM9h/wDv5LIStLXhjK/S8p4mGBVON8036rj6ktDeikwGFNR7GAwXECTt1Psm5dsErBY8h8vGs573khgIA7ka+116GOFeHGUS3r27qy4Nw9lGk2mwQAPck3JPeSjy20dlq+FNZ2NcrWjNYp+VpHRYvGed7ndSTKv+N4rzvpg6Egn20+aochgCNd1yqLtj5HdIFpYa8qDEsdtBurxtIBgEKsbTJcRBsrZD0QmkIEld8rW21KJGHBaUIyh5ydgEqFsTaTnRGv4JPYS4ibiy7hn5Sb3UrGfaPcosLAPDd1SRXjDskpsLRZOy5p2At6phY4X13soalBpEOdMaAdUTTpwASU78E8AeOY98ZbRclJjIZLrkHXsj2vF56dIlQCsxxiPKQhO0CyDOxT5GQW6rj6paQALm8o52Roa0aaJ2IcA3L133SavYmrIW1IcCRqNFI6mbAm0yY2GyFrNI8wM2hcw1Qxc/EIgpppDqjmOZNgNxB6qYPDQDG111gmGkjsigWFhYRfcpoa/sDxFAkAt3Ejsm0BaHbI+ocryGXsIH1KHqSLRrc9ymxNHK1VobF5GnSUPSbml3TVKrTJiGz1SwRnMGjVTkYmOBGvmCcxs63Kc+kGuEtk9dkZSoZrsItr3TX0CZS1aJY8dDcHv0K9J5W4eGMOaC4w70svO+MVXCm5wjyOafuN16by2ZoMeT8QmT02W/Ck7sJSdIF504U2rhK8Nl7WF7Y1LmDMBbUkAj3Xj+AJeMoBJGkX/7XvzKrSYDgT6rEcxcuspVRXpeRr3f8jRGUO/qaNpOo6p8kE8oSk9Hn7qBGv6q85RwZfiaZIsCXH2B/RS4pmYg6iTeNI0Wo5CwZyvqOGrsrf8AEakep+ijjjchtmybog+L4o06T3jZpj1hGtCoObHODGR8JfleOxEj5hdL0QYfEvIaHmS57jJ9bk/RPfWOUEjQW/fsiuN0w0sDfhAJM9SqmpUdodtLLhSawzWT7Owo4kua0gqKnWIJFpO/Zcp0BldJhxiFG6mdBroTO6b+xaHY5xbGUz1UNFtySNpKTKLgMsiZTcNVcHunTf0CS/slHXsBEgKei5rmEdFHVuCR8M2UT3Whvb37JWCwxuXskjG0XwPKkn1GOZTJIaPsiSVM5pI8okk23v0TGvJGZrSLkO7jpKcxjpgyA3zAQpik0Rs5iGuiLE7t6dVXV6TgczB0kKw8W0kOBOi7lJhgudZ7I65GmBeLmcGvERcdlNXJJ0kDdS0cK0ODyPNJEnS3ULrKbs+YeYGTA09AihtWyZlItYHGJ1ynYIFkuu34h5vWeiMqsc5jnEXFo0tuhKbHAtY0AWtPT1T14JANZ7/EzQQALd1aNcSJLdQCnNfmIkD23jYItktGVwEuNwNh0lLDHvYJQxOd0htxA/RTPpPnQEjYnRDswuXOWzBNvVcwdUvBaTBbr39U442LQTWeAC82AOUDqYuoWGAMgOZ2oAn9hTupjKM1xJ9j1hLh77Py7yJjZV6CYBXD3ENIOt/RdoPcxttydFY4t4BY0C9iSO43QviDxcjT8IMnp1RVDA6+Fzse12pY6B0sYPrKIxvNQo0WU2vAaKTSASPhDToNzYe0qUsJzECQ5sT0jqsVx7hDzVaARGRzBmmMjiTY9QSR6ALTjlVjLvB/+SqjmEljWFrmhzgZsS6C1ojTyg3KvsJzd/EANcWkOny3u4CWuA2vIhYXhnJpMB7iWnUC3uDC1OD4cykcrGRlEAm5deZLlU31Vh2VFg+sfOyLOEgdFt+UCf4ZhdYmY/xBIb8gF589rg62oMQfxWw5e4kxwYxpBjM0xMBwixP+rrdkcLy7E8mtBVNzU5ow7i7QFpHrmH6qehiTlcbE+YtA94H4LPczcWY+gaYcDUztGQXJElaSlgHGjMYnGF5G4M6bKTxRkDRBMx3ChqUpbPwlsTFtUxtGCHToR7rktpheAhxizm6iL7nZVbKD2u1Nzp0V3iCHDPN7QNRG6GZiQ55tobH6pyBjXUjAgibn23lDeH10Fz3CmL5e63l0/ROd5hMRt6KW1QXaAKeKaXuYJLdk3MWlvc2PvuiDRIJbF5F46p9TDicua4+UpJWGy3ZiLC40C6qvKz+o/ckrt/Y8ljiWwbW1+pQ+GeSDJJnXvdJJJaJZeY6g3yCBAbooOL0gDTIES38l1JOWmUAvYIIjdR4Wzvl8kkllHz/RLQ2qLn0H1T8WwZGmL6T/AKlJJOQIFw7bjtP0CN3aerSuJKVofgqf2RsSbKPFMANhrKSSa2J6HYNsi9/N9FC+zXRbyn6pJKvCfoJpsGVhi5YPqVVlgFV9tXD6JJKXtif7FjR+A93ge3RRcwYZnggxcPsekriSqO0UN4V8KO+27s0JJLfm/Vf9BbG1qLYeYvDbyVk34t9LHUKdNxYx1RxLW2Bza/U/eupJQ2UjR4jidVlSnlqESwk6X/5ag/Afcp6dIZ3GLnKSdz5Ekkp/qUypYJeJ3cQe6Kx9qrgNMuiSSyWjN6Ez4PcKFjfID/e76lJJNAywawZXW3UlKmC15jqfkkkmg8BKHxj0/BQYqg3MDH2ep6pJJS0xouaOGblFth16JJJKCj//2Q=="
}
```

### Configure API Gateway 
<a name="api-gateway"></a>
* Finally, we need an API to have the service accessible
* This API accepts image POST to it and pass the image to ```invoke_image_object_detection```

```curl -X POST -H 'content-type: application/octet-stream' --data-binary @$f $OD_API | jq .``` 

* we can create it by console also by aws cli 

In [41]:
!aws lambda add-permission \
    --function-name invoke_endpoint \
    --action lambda:InvokeFunction \
    --statement-id apigateway \
    --principal apigateway.amazonaws.com    


An error occurred (ResourceConflictException) when calling the AddPermission operation: The statement id (apigateway) provided already exists. Please provide a new statement id, or remove the existing statement.


In [42]:
!sed "s/<account_id>/$account_id/g" latestswagger2-template.json > latestswagger2-tmp.json 
!sed "s/<region>/$region/g" latestswagger2-tmp.json > latestswagger2.json 

In [43]:
api_info = !aws apigateway import-rest-api \
    --fail-on-warnings \
    --body 'file:////home/ec2-user/SageMaker/incremental-training-mlops/01-byoc/latestswagger2.json' 

In [44]:
api_info

['{',
 '    "id": "pdtujuf6u4",',
 '    "name": "audio_0529",',
 '    "createdDate": 1626444300,',
 '    "version": "2021-05-29T14:33:28Z",',
 '    "binaryMediaTypes": [',
 '        "application/octet-stream"',
 '    ],',
 '    "apiKeySource": "HEADER",',
 '    "endpointConfiguration": {',
 '        "types": [',
 '            "EDGE"',
 '        ]',
 '    },',
 '    "disableExecuteApiEndpoint": false',
 '}']

In [45]:
api_obj = json.loads(''.join(api_info))
api_id = api_obj['id']
api_id

'pdtujuf6u4'

In [42]:
# OLD 'rbrdok3cva'
# api_obj = json.loads(''.join(api_info))
# api_id = api_obj['id']
# api_id

'rbrdok3cva'

In [49]:
# !aws apigateway create-deployment --rest-api-id $api_id --stage-name dev
!aws apigateway create-deployment --rest-api-id pdtujuf6u4 --stage-name dev

{
    "id": "ck5oh9",
    "createdDate": 1626444488
}


### Manually Setup API-Gateway in Console
<a name="binary-content"></a>


####  Create Restful API 
<img src="../03-lambda-api/content_image/build_api_gateway.png" alt="rest_api" width="600"/>

#### Create resource and methods 
* click the drop down manual and name your resource 
<img src="../03-lambda-api/content_image/create_resource.png" alt="create_resource" width="600"/>

* focus on the resource just created, click the drop down manual and select create method, then select backend lambda function 
<img src="../03-lambda-api/content_image/create_method.png" alt="create_method" width="600"/>
<img src="../03-lambda-api/content_image/create_method2.png" alt="create_method" width="600"/>


### Configurations for passing the binary content to backend
* Add binary media type in ```Settings``` of this API 
* Configure which attribute to extract and fill it in event in Lambda integration

<img src="../03-lambda-api/content_image/setting.png" alt="binary_media" width="600"/>
<img src="../03-lambda-api/content_image/integration.png" alt="config lambda function" width="600"/>
<img src="../03-lambda-api/content_image/setting1.png" alt="config lambda function2" width="600"/>

### Test API Gateway 
<a name="test-api"></a>

In [130]:
api_endpoint = "https://{}.execute-api.{}.amazonaws.com/dev/classify".format(api_id, region)

!curl -X POST -H 'content-type: application/octet-stream'  --data-binary @./train_data/train_00002.wav $api_endpoint


{"label": 0, "probability": [0.9513887763023376, 0.00023666038759984076, 0.04708951339125633, 9.738496009958908e-05, 6.840967398602515e-05, 5.344108558347216e-06, 4.340481973486021e-05, 0.00042902593850158155, 1.1892406291735824e-05, 0.0006295717321336269]}

In [131]:
api_endpoint

'https://pdtujuf6u4.execute-api.us-west-2.amazonaws.com/dev/classify'

In [67]:
api_endpoint

'https://pdtujuf6u4.execute-api.us-west-2.amazonaws.com/dev/classify'

In [53]:
endpoint_name = new_endpoint_name
lambda_role_arn = "arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole"
# print(lambda_role_arn)
model_s3_path = model.model_data
%store endpoint_name 
%store lambda_role_arn
%store model_s3_path 

Stored 'endpoint_name' (str)
Stored 'lambda_role_arn' (str)
Stored 'model_s3_path' (str)


In [54]:
lambda_role_arn

'arn:aws:iam::355444812467:role/AmazonSageMaker-LambdaExecutionRole'

In [28]:
api_endpoint = "https://runtime.sagemaker.us-west-2.amazonaws.com/endpoints/audio-1626202830/invocations"
token = "0B22878B03FE197EF8D6"
!curl -X POST -H 'content-type: application/octet-stream' --data-binary @./train_data/train_00002.wav    $api_endpoint

{"message":"Missing Authentication Token"}

In [64]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./train_data/train_00077.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 0, "probability": [0.998103141784668, 4.8802314267959446e-05, 8.627666102256626e-05, 3.7545803479588358e-06, 1.8969452639794326e-06, 2.4744387701503e-05, 7.638564966327976e-06, 6.144330768620421e-07, 1.734519173623994e-06, 0.0017214053077623248]}

In [140]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_01581.wav \
'https:///pdtujuf6u4.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 9, "probability": [8.059015090111643e-05, 5.81457388761919e-05, 0.00019981291552539915, 0.0001927862031152472, 6.296393985394388e-05, 0.000279656465863809, 0.0002843793190550059, 2.493889769539237e-05, 0.0008642131579108536, 0.9979525804519653]}

In [132]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_01581.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 9, "probability": [8.059015090111643e-05, 5.81457388761919e-05, 0.00019981291552539915, 0.0001927862031152472, 6.296393985394388e-05, 0.000279656465863809, 0.0002843793190550059, 2.493889769539237e-05, 0.0008642131579108536, 0.9979525804519653]}

In [138]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./train_data/train_00002.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 0, "probability": [0.9513887763023376, 0.00023666038759984076, 0.04708951339125633, 9.738496009958908e-05, 6.840967398602515e-05, 5.344108558347216e-06, 4.340481973486021e-05, 0.00042902593850158155, 1.1892406291735824e-05, 0.0006295717321336269]}

In [134]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00002.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 0, "probability": [0.9513887763023376, 0.00023666038759984076, 0.04708951339125633, 9.738496009958908e-05, 6.840967398602515e-05, 5.344108558347216e-06, 4.340481973486021e-05, 0.00042902593850158155, 1.1892406291735824e-05, 0.0006295717321336269]}

In [57]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00002.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"label": 0, "probability": [0.9513887763023376, 0.00023666038759984076, 0.04708951339125633, 9.738496009958908e-05, 6.840967398602515e-05, 5.344108558347216e-06, 4.340481973486021e-05, 0.00042902593850158155, 1.1892406291735824e-05, 0.0006295717321336269]}

In [58]:
print(datetime.now(timezone(timedelta(hours=8))))
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00001.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'
print(datetime.now(timezone(timedelta(hours=8))))

2021-07-16 22:24:49.098558+08:00
{"label": 0, "probability": [0.9903320670127869, 0.0001989333686651662, 0.0007428666576743126, 0.00016775673429947346, 3.222770828870125e-05, 3.8894515455467626e-05, 0.0003140326589345932, 0.0003089535457547754, 3.332734195282683e-05, 0.007830953225493431]}2021-07-16 22:24:50.366420+08:00
